In [ ]:
import requests

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
import requests

url = "https://api.foursquare.com/v3/places/search?radius=1000"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3XRUAarAkYkZaUxo7U14qSYIS91CYwbt1bF94/FymeqU="
}

response = requests.get(url, headers=headers)

print(response.text)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
import requests

headers = {
    "accept": "application/json",
    "Authorization": "fsq3XRUAarAkYkZaUxo7U14qSYIS91CYwbt1bF94/FymeqU="
}

url = "https://api.foursquare.com/v3/places/search"
params = {
    'radius': 1000,
    'll': '40.7243,-74.0018',
    'query': 'coffee',
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    print(response.text)

else:
    print(f"Error: {response.status_code}, {response.text}")

Put your parsed results into a DataFrame

In [ ]:
data = response.json()
    
for place in data['results']:
    name = place.get('name', 'N/A')
    address = place.get('location', {}).get('formatted_address', 'N/A')
    categories = [category.get('name', 'N/A') for category in place.get('categories', [])]
        
    print(f"Name: {name}")
    print(f"Address: {address}")
    print(f"Categories: {', '.join(categories)}")
    print("-----")

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
import requests

url = "https://api.yelp.com/v3/businesses/search"
params = {
    'radius' : 1000
}

headers = {
    "accept": "application/json",
    "Authorization": "oCE9nQh4V-FyXHEU0GDnZV-cdmqUWjYgoS_7OkG7XEEi8KDgtqVRZlYkcQn_q94PrjNaIhgq7nfLWb0wNk2wn_2RNBjm_lm4vGIkkWW6OhQ9xru_q3QRjywgg6myZnYx"
}

response = requests.get(url, headers=headers)

print(response.text)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
url = "https://api.yelp.com/v3/businesses/search"
params = {
    'radius' : 1000,
    'latitude': 40.7243,
    'longitude' : -74.0018,
    'query': 'coffee',
}

headers = {
    "accept": "application/json",
    "Authorization": "oCE9nQh4V-FyXHEU0GDnZV-cdmqUWjYgoS_7OkG7XEEi8KDgtqVRZlYkcQn_q94PrjNaIhgq7nfLWb0wNk2wn_2RNBjm_lm4vGIkkWW6OhQ9xru_q3QRjywgg6myZnYx"
}

response_yelp = requests.get(url, headers=headers)

print(response_yelp.text)

Put your parsed results into a DataFrame

In [ ]:
data_yelp = response_yelp.json()
    
for place in data_yelp['results']:
    name = place.get('name', 'N/A')
    address = place.get('location', {}).get('formatted_address', 'N/A')
    categories = [category.get('name', 'N/A') for category in place.get('categories', [])]
        
    print(f"Name: {name}")
    print(f"Address: {address}")
    print(f"Categories: {', '.join(categories)}")
    print("-----")


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating